## `Pastikan klik 'File > Save a copy in Drive' sebelum memulai pengerjaan`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import BDay
import datetime as dt

#Dataset

Data yang digunakan adalah data yang berasal dari Tokopedia (***bukan data sesungguhnya***). Mengenai penjelasan dataset adalah sebagai berikut:

|variable                       |class     |description |
|:------------------------------|:---------|:-----------|
**order_detail:**
id 			|object| angka unik dari order / id_order
customer_id 		|object|angka unik dari pelanggan
order_date 		|object| tanggal saat dilakukan transaksi
sku_id 			|object| angka unik dari produk (sku adalah stock keeping unit)
price			|int64| harga yang tertera pada tagging harga
qty_ordered 		|int64| jumlah barang yang dibeli oleh pelanggan
before_discount	|float64| nilai harga total dari produk (price * qty_ordered)
discount_amount	|float64| nilai diskon product total
after_discount		|float64| nilai harga total produk ketika sudah dikurangi dengan diskon
is_gross 		|int64| menunjukkan pelanggan belum membayar pesanan
is_valid		|int64| menunjukkan pelanggan sudah melakukan pembayaran
is_net			|int64| menunjukkan transaksi sudah selesai
payment_id 		|int64| angka unik dari metode pembayaran
||
**sku_detail:**
id |object| angka unik dari produk (dapat digunakan untuk key saat join)
sku_name 		|object| nama dari produk
base_price		|float64| harga barang yang tertera pada tagging harga / price
cogs 			|int64| cost of goods sold / total biaya untuk menjual 1 produk
category		|object| kategori produk
||
**customer_detail:**
id 			|object| angka unik dari pelanggan
registered_date	|object| tanggal pelanggan mulai mendaftarkan diri sebagai anggota
||
**payment_detail:**
id			|int64| angka unik dari metode pembayaran
payment_method	|object| metode pembayaran yang digunakan


In [ ]:
#Sumber data yang digunakan
path_od = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/order_detail.csv"
path_pd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/payment_detail.csv"
path_cd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/customer_detail.csv"
path_sd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/sku_detail.csv"
df_od = pd.read_csv(path_od)
df_pd = pd.read_csv(path_pd)
df_cd = pd.read_csv(path_cd)
df_sd = pd.read_csv(path_sd)

In [ ]:
#Mengampilkan 5 baris pertama
df_od.head()

id customer_id  order_date   sku_id    price  qty_ordered  \
0  ODR9939707760w    C713589L  2021-11-19  P858068    26100          200   
1  ODR7448356649d    C551551L  2021-11-19  P886455  1971942            5   
2  ODR4011281866z    C685596L  2021-11-25  P678648  7482000            1   
3  ODR3378927994s    C830683L  2021-11-22  P540013  3593680            1   
4  ODR4904430099k    C191766L  2021-11-21  P491032  4413220            1   

   before_discount  discount_amount  after_discount  is_gross  is_valid  \
0        5220000.0       2610000.00      2610000.00         1         1   
1        9859710.0       2464927.50      7394782.50         1         0   
2        7482000.0       2065344.62      5416655.38         1         0   
3        3593680.0       1455440.40      2138239.60         1         1   
4        4413220.0       1059172.80      3354047.20         1         1   

   is_net  payment_id  
0       0           5  
1       0           5  
2       0           4  
3       1           5  
4       1           4

In [ ]:
#Mengampilkan 5 baris pertama
df_pd.head()

id  payment_method
0   1             cod
1   2     jazzvoucher
2   3  customercredit
3   4         Payaxis
4   5      jazzwallet

In [ ]:
#Mengampilkan 5 baris pertama
df_cd.head()

id registered_date
0  C996508L      2021-07-10
1  C180415L      2021-07-18
2  C535451L      2021-07-23
3  C177843L      2021-07-12
4  C951682L      2021-07-27

In [ ]:
#Mengampilkan 5 baris pertama
df_sd.head()

id                  sku_name  base_price     cogs     category
0  P798444                 AT-FSM-35    57631.70    46052  Kids & Baby
1  P938347           AYS_Haier-18HNF  3931789.26  3499256   Appliances
2  P826364  Atalian _DV206A-Brown-41   324597.00   243426  Men Fashion
3  P467533  Darul_Sakoon_Food_Bundle     2870.42     2378   Superstore
4  P229955   HP_15AY-15-Ay072NIA-ci3  2265625.00  1631250    Computing

In [ ]:
#Menjalankan SQL di Colab
from sqlite3 import connect
conn = connect(':memory:')
df_od.to_sql('order_detail',conn, index=False, if_exists='replace')
df_pd.to_sql('payment_detail', conn, index=False, if_exists='replace')
df_sd.to_sql('sku_detail', conn, index=False, if_exists='replace')
df_cd.to_sql('customer_detail', conn, index=False, if_exists='replace')

3998

In [ ]:
#Query SQL untuk menggabungkan data
df = pd.read_sql("""
SELECT
    order_detail.*,
    payment_detail.payment_method,
    sku_detail.sku_name,
    sku_detail.base_price,
    sku_detail.cogs,
    sku_detail.category,
    customer_detail.registered_date
FROM order_detail
LEFT JOIN payment_detail
    on payment_detail.id = order_detail.payment_id
LEFT JOIN sku_detail
    on sku_detail.id = order_detail.sku_id
LEFT JOIN customer_detail
    on customer_detail.id = order_detail.customer_id
""", conn)

In [ ]:
#Mengampilkan 5 baris pertama
df.head()

id customer_id  order_date   sku_id    price  qty_ordered  \
0  ODR9939707760w    C713589L  2021-11-19  P858068    26100          200   
1  ODR7448356649d    C551551L  2021-11-19  P886455  1971942            5   
2  ODR4011281866z    C685596L  2021-11-25  P678648  7482000            1   
3  ODR3378927994s    C830683L  2021-11-22  P540013  3593680            1   
4  ODR4904430099k    C191766L  2021-11-21  P491032  4413220            1   

   before_discount  discount_amount  after_discount  is_gross  is_valid  \
0        5220000.0       2610000.00      2610000.00         1         1   
1        9859710.0       2464927.50      7394782.50         1         0   
2        7482000.0       2065344.62      5416655.38         1         0   
3        3593680.0       1455440.40      2138239.60         1         1   
4        4413220.0       1059172.80      3354047.20         1         1   

   is_net  payment_id payment_method  \
0       0           5     jazzwallet   
1       0           5     jazzwallet   
2       0           4        Payaxis   
3       1           5     jazzwallet   
4       1           4        Payaxis   

                                            sku_name  base_price     cogs  \
0                      RB_Dettol Germ Busting Kit-bf     26100.0    18270   
1                                     PS4_Slim-500GB   1971942.0  1321182   
2  Changhong Ruba 55 Inches UD55D6000i Ultra HD T...   7482000.0  5162580   
3                               dawlance_Inverter 30   3593680.0  3054628   
4                       Dawlance_Inverter-45 2.0 ton   4413220.0  3177472   

        category registered_date  
0         Others      2021-07-07  
1  Entertainment      2021-11-20  
2  Entertainment      2021-11-19  
3     Appliances      2021-11-03  
4     Appliances      2021-07-05

In [ ]:
#Menampikan tipe data tiap kolom
df.dtypes

id                  object
customer_id         object
order_date          object
sku_id              object
price                int64
qty_ordered          int64
before_discount    float64
discount_amount    float64
after_discount     float64
is_gross             int64
is_valid             int64
is_net               int64
payment_id           int64
payment_method      object
sku_name            object
base_price         float64
cogs                 int64
category            object
registered_date     object
dtype: object

In [ ]:
#Mengubah tipe data agar mudah dilakukan pengolahan data
df = df.astype({"before_discount":'int', "discount_amount":'int', "after_discount":'int',"base_price":'int'})
df.dtypes

id                 object
customer_id        object
order_date         object
sku_id             object
price               int64
qty_ordered         int64
before_discount     int64
discount_amount     int64
after_discount      int64
is_gross            int64
is_valid            int64
is_net              int64
payment_id          int64
payment_method     object
sku_name           object
base_price          int64
cogs                int64
category           object
registered_date    object
dtype: object

In [ ]:
#Mengubah tipe kolom Date menjadi Datetime
df['order_date']= pd.to_datetime(df['order_date'])
df['registered_date']= pd.to_datetime(df['registered_date'])
df.dtypes


id                         object
customer_id                object
order_date         datetime64[ns]
sku_id                     object
price                       int64
qty_ordered                 int64
before_discount             int64
discount_amount             int64
after_discount              int64
is_gross                    int64
is_valid                    int64
is_net                      int64
payment_id                  int64
payment_method             object
sku_name                   object
base_price                  int64
cogs                        int64
category                   object
registered_date    datetime64[ns]
dtype: object

## `No 1`
**Dear Data Analyst**,
<br></br>
Akhir tahun ini, perusahaan akan memberikan hadiah bagi pelanggan yang memenangkan kompetisi **Festival Akhir Tahun**. Tim Marketing membutuhkan bantuan untuk menentukan perkiraan hadiah yang akan diberikan pada pemenang kompetisi nantinya. Hadiah tersebut akan diambil dari **TOP 5 Produk** dari Kategori **Mobiles & Tablets** selama tahun 2022, dengan jumlah kuantitas penjualan (valid = 1) paling tinggi.

Mohon bantuan, untuk mengirimkan data tersebut sebelum akhir bulan ini ke Tim Marketing. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Marketing**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Memfilter data dengan valid = 1
2. Memfilter data dengan Category = Mobiles & Tablets
3. Memfilter data pada transaksi selama 2022
4. Gunakan groupby berdasarkan sku_name
5. Gunakan sort_values untuk mengurutkan data
6. Gunakan head untuk menampilkan top 5 produk


```



</details>

## `Jawaban No 1`

In [ ]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
data1 = df[(df['is_valid'] == 1) & (df['category']=='Mobiles & Tablets') & (df['order_date'].dt.year == 2022)]
data1 = data1.groupby(['sku_name'])['qty_ordered'].sum().reset_index(name='qty_2022').sort_values(['qty_2022'], ascending=False).head()
data1

sku_name  qty_2022
1               IDROID_BALRX7-Gold      1000
2         IDROID_BALRX7-Jet black         31
3               Infinix Hot 4-Gold        15
43  samsung_Grand Prime Plus-Black        11
34             infinix_Zero 4-Grey        10

## `No 2`
**Dear Data Analyst**,
<br></br>
Menindaklanjuti meeting gabungan Tim Werehouse dan Tim Marketing, kami menemukan bahwa ketersediaan stock produk dengan Kategori Others pada akhir 2022 kemarin masih banyak.
1. Kami mohon bantuan untuk melakukan pengecekan data penjualan kategori tersebut dengan tahun 2021 secara kuantitas penjualan. Dugaan sementara kami, telah terjadi penurunan kuantitas penjualan pada 2022 dibandingkan 2021. (Mohon juga menampilkan data ke-15 kategori)
<br></br>
2. Apabila memang terjadi penurunan kuantitas penjualan pada kategori Others, kami mohon bantuan untuk menyediakan data TOP 20 nama produk yang mengalami penurunan paling tinggi pada 2022 jika dibanding dengan 2021. Hal ini kami gunakan sebagai bahan diskusi pada meeting selanjutnya.  

Mohon bantuan untuk mengirimkan data tersebut paling lambat 4 hari dari hari ini. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Werehouse**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Memfilter data dengan valid = 1
2. Memfilter data pada transaksi selama 2021
3. Memfilter data pada transaksi selama 2022
4. Gunakan groupby berdasarkan sku_name masing-masing tahun
5. Menggabungkan kedua data dengan merge
6. Lakukan pengurangan kolom qty 2022 dengan qty 2021


```



</details>

## `Jawaban No 2.1`

In [ ]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
data2_21 = df[(df['is_valid'] == 1) & (df['order_date'].dt.year == 2021)].groupby(['category'])['qty_ordered'].sum().reset_index(name='qty_2021').sort_values(['qty_2021'], ascending=False)
data2_21.head()

category  qty_2021
12        Soghaat       759
10         Others       426
13     Superstore       327
8     Men Fashion       237
6   Home & Living       193

In [ ]:
data2_22 = df[(df['is_valid'] == 1) & (df['order_date'].dt.year == 2022)].groupby(['category'])['qty_ordered'].sum().reset_index(name='qty_2022').sort_values(['qty_2022'], ascending=False)
data2_22.head()

category  qty_2022
9   Mobiles & Tablets      1154
12            Soghaat       612
13         Superstore       536
14      Women Fashion       489
10             Others       263

In [ ]:
data2_merge = pd.merge(data2_21, data2_22, on='category', how='left')
data2_merge['growth'] = data2_merge['qty_2022'] - data2_merge['qty_2021']
data2_merge = data2_merge.sort_values(by=['growth'], ascending=True)
data2_merge

category  qty_2021  qty_2022  growth
1               Others       426       263    -163
0              Soghaat       759       612    -147
3          Men Fashion       237       175     -62
9    Beauty & Grooming       168       153     -15
7                Books       171       195      24
11          Appliances       124       148      24
6      Health & Sports       173       200      27
12           Computing       109       153      44
5   School & Education       184       237      53
4        Home & Living       193       250      57
8          Kids & Baby       170       227      57
14       Entertainment        77       150      73
2           Superstore       327       536     209
10       Women Fashion       140       489     349
13   Mobiles & Tablets       107      1154    1047

## `Jawaban No 2.2`

In [ ]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
others_21 = df[(df['is_valid'] == 1) & (df['category']=='Others') & (df['order_date'].dt.year == 2021)].groupby(['sku_name'])['qty_ordered'].sum().reset_index(name='qty_2021').sort_values(['qty_2021'], ascending=False)
others_21.head(15)

sku_name  qty_2021
39                      RB_Dettol Germ Busting Kit-bf       200
43                                Telemall_MM-DR-HB-L        23
11                           Dawlance_MD 10 + DWB 600        23
69             iu_Tickets General Enclosure-Islamabad        20
40                     RS_Rehmat-e-Shereen Mix Mithai        13
73                                    kansai_NeverWet        10
80                        sindbad_Sindbad Gift Card-3         7
66                                         emart_00-1         7
50                                     Vouch 365 2016         5
67                    emart_Tyre Shape Air Compressor         5
2                                    Am-PTV_ATS-004-M         5
44  The Vitamin Company Kojic Acid Whitening Cream...         4
62                                 duma_4561253300294         4
26                                     MEGUIAR_G12711         4
81                         sockoye_QG in Quarter Grey         4

In [ ]:
others_22 = df[(df['is_valid'] == 1) & (df['category']=='Others') & (df['order_date'].dt.year == 2022)].groupby(['sku_name'])['qty_ordered'].sum().reset_index(name='qty_2022').sort_values(['qty_2022'], ascending=False)
others_22.head(15)

sku_name  qty_2022
37                  RB_Dettol Germ Busting Kit-bf        45
82                emart_Tyre Shape Air Compressor        34
43                                  SKMT_Medicine        20
90                     shoppers stop_MagicGlass-3        13
47                TCF_Educate-a-Child-for-a-Month         8
80                                     emart_0-37         7
9      Entertainer Asia_Vouch 365-2017 BookLahore         7
85                    kansai_Undercoating Aerosol         6
61  aw_AIR MAX MOISTURE ABSORBER & REFILLER-450gm         5
88                                proactiv_acne_1         4
50                                 Tmall_Autocool         3
44          Saylani_Health-Contribution-Package-1         3
64                     aw_CONSTRUCTION FOAM-500ml         3
38            Rajesh_Blue Car Emergency Tow Chain         3
91                    sindbad_Sindbad Gift Card-1         3

In [ ]:
others_merge = pd.merge(others_21, others_22, on='sku_name', how='outer')
others_merge.fillna(0, inplace=True)
others_merge['growth'] = others_merge['qty_2022'] - others_merge['qty_2021']
others_merge = others_merge.sort_values(by=['growth'], ascending=True)
others_merge.head(20)

sku_name  qty_2021  qty_2022  \
69                       RB_Dettol Germ Busting Kit-bf     200.0      45.0   
18                            Dawlance_MD 10 + DWB 600      23.0       0.0   
84                                 Telemall_MM-DR-HB-L      23.0       2.0   
137             iu_Tickets General Enclosure-Islamabad      20.0       0.0   
70                      RS_Rehmat-e-Shereen Mix Mithai      13.0       0.0   
141                                    kansai_NeverWet      10.0       1.0   
154                        sindbad_Sindbad Gift Card-3       7.0       0.0   
133                                         emart_00-1       7.0       1.0   
93                                      Vouch 365 2016       5.0       0.0   
4                                     Am-PTV_ATS-004-M       5.0       0.0   
129                                 duma_4561253300294       4.0       0.0   
155                         sockoye_QG in Quarter Grey       4.0       0.0   
85   The Vitamin Company Kojic Acid Whitening Cream...       4.0       0.0   
115                     aw_Octane Booster-12oz./354ml        3.0       0.0   
43                                      MEGUIAR_G12711       4.0       1.0   
88                                      Trans2_LW 999        3.0       0.0   
50                                      MEGUIAR_G19216       2.0       0.0   
26                                        JBS_IFAM-009       2.0       0.0   
60                                     MEGUIAR_X1030EU       2.0       0.0   
156                         sstop_Universallensclipkit       2.0       0.0   

     growth  
69   -155.0  
18    -23.0  
84    -21.0  
137   -20.0  
70    -13.0  
141    -9.0  
154    -7.0  
133    -6.0  
93     -5.0  
4      -5.0  
129    -4.0  
155    -4.0  
85     -4.0  
115    -3.0  
43     -3.0  
88     -3.0  
50     -2.0  
26     -2.0  
60     -2.0  
156    -2.0

## `No 3`
**Dear Data Analyst**,
<br></br>
Terkait ulang tahun perusahaan pada 2 bulan mendatang, Tim Digital Marketing akan memberikan informasi promo bagi pelanggan pada akhir bulan ini. Kriteria pelanggan yang akan kami butuhkan adalah mereka yang sudah melakukan check-out namun belum melakukan pembayaran (is_gross = 1) selama tahun 2022. Data yang kami butuhkan adalah ID Customer dan Registered Date.

Mohon bantuan, untuk mengirimkan data tersebut sebelum akhir bulan ini ke Tim Digital Marketing. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Digital Marketing**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Memfilter data dengan gross = 1
2. Memfilter data dengan valid = 0
3. Memfilter data dengan net = 0
4. Memfilter data pada transaksi selama 2022


```



</details>

## `Jawaban No 3`

In [ ]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
data3 = df[(df['is_gross']==1)&(df['is_valid']==0)&(df['is_net']==0)&(df['order_date'].dt.year==2022)]
data3 = data3[['customer_id','registered_date']].sort_values(by=['registered_date'], ascending=True)
data3.drop_duplicates(inplace=True)
data3

customer_id registered_date
4168    C349980L      2021-07-02
3021    C849967L      2021-07-02
1304    C378684L      2021-07-03
606     C831683L      2021-07-03
1331    C893754L      2021-07-03
...          ...             ...
2335    C877563L      2022-08-24
157     C788348L      2022-08-25
3512    C741725L      2022-08-25
315     C925840L      2022-08-28
443     C466867L      2022-08-28

[820 rows x 2 columns]

In [ ]:
#Jalankan kode ini untuk mendownload file
from google.colab import files
data3.to_csv('audience_list.csv', encoding = 'utf-8-sig',index=False) #ganti [nama variabel file] dengan nama variabel yang digunakan
files.download('audience_list.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## `No 4`
**Dear Data Analyst**,
<br></br>
Pada bulan October hingga Desember 2022, kami melakukan campaign setiap hari Sabtu dan Minggu. Kami hendak menilai, apakah campaign tersebut cukup berdampak pada kenaikan penjualan (before_discount). Mohon bantuan untuk menampilkan data:
1. Rata-rata harian penjualan weekends (Sabtu dan Minggu) vs rata-rata harian penjualan weekdays (Senin-Jumat) per bulan tersebut. Apakah ada peningkatan penjualan pada masing-masing bulan tersebut.
2. Rata-rata harian penjualan weekends (Sabtu dan Minggu) vs rata-rata harian penjualan weekdays (Senin-Jumat) keseluruhan 3 bulan tersebut.

Mohon bantuan untuk mengirimkan data tersebut paling lambat minggu depan. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Campaign**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Buatlah kolom baru untuk day, month, dan month number
2. Memfilter data dengan valid = 1
3. Memfilter data dengan day termasuk 'Saturday', 'Sunday' dengan menggunakan fungsi isin dan simpan dalam variable 'weekends'
4. Memfilter data dengan day termasuk 'Monday','Tuesday','Wednesday','Thusday','Friday' dengan menggunakan fungsi isin dan simpan dengan variabel 'weekday'
5. Memfilter data pada transaksi selama 2022 bulan Oktober sampai dengan Desember
6. Gunakan groupby berdasarkan pada masing-masing variabel
7. Menggabungkan kedua data dengan merge
8. Tampilkan pada grafik batang untuk melihat perbedaananya


```



</details>

## `Jawaban No 4.1`

In [ ]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
df['day'] = df['order_date'].dt.day_name()
df['month'] = df['order_date'].dt.month_name()
df['month_number'] = df['order_date'].dt.month
data4 = df[(df['is_valid']==1)&(df['month_number'].isin([10,11,12]))&(df['order_date'].dt.year==2022)]

In [ ]:
weekend = data4[(data4['day'].isin(['Saturday','Sunday']))].groupby(['day','month','month_number'])['before_discount'].sum().reset_index(name='weekend')
weekend

day     month  month_number   weekend
0  Saturday  December            12   2121234
1  Saturday  November            11   6545822
2  Saturday   October            10  14891674
3    Sunday  December            12   6090754
4    Sunday  November            11   5002268
5    Sunday   October            10   2233348

In [ ]:
weekday = data4[(data4['day'].isin(['Monday','Tuesday','Wednesday','Thursday','Friday']))].groupby(['day','month','month_number'])['before_discount'].sum().reset_index(name='weekday')
weekday

day     month  month_number   weekday
0      Friday  December            12   2678962
1      Friday  November            11  11829274
2      Friday   October            10   7889218
3      Monday  December            12   2699668
4      Monday  November            11   3706084
5      Monday   October            10   5111076
6    Thursday  December            12  27756190
7    Thursday  November            11   4296930
8    Thursday   October            10   4050430
9     Tuesday  December            12   9767606
10    Tuesday  November            11  16613926
11    Tuesday   October            10  12259634
12  Wednesday  December            12   8352754
13  Wednesday  November            11    781782
14  Wednesday   October            10  10050704

In [ ]:
avg_weekend = weekend.groupby(['month','month_number'])['weekend'].mean().reset_index(name='avg_weekend')
avg_weekend

month  month_number  avg_weekend
0  December            12    4105994.0
1  November            11    5774045.0
2   October            10    8562511.0

In [ ]:
avg_weekday = weekday.groupby(['month','month_number'])['weekday'].mean().reset_index(name='avg_weekday')
avg_weekday

month  month_number  avg_weekday
0  December            12   10251036.0
1  November            11    7445599.2
2   October            10    7872212.4

In [ ]:
avg_merge = pd.merge(avg_weekend, avg_weekday, on='month', how='left').sort_values(by=['month_number_x'], ascending=True)
avg_merge = avg_merge[['month','avg_weekday','avg_weekend']]
avg_merge['growth'] = avg_merge['avg_weekend'] - avg_merge['avg_weekday']
avg_merge

month  avg_weekday  avg_weekend     growth
2   October    7872212.4    8562511.0   690298.6
1  November    7445599.2    5774045.0 -1671554.2
0  December   10251036.0    4105994.0 -6145042.0

In [ ]:
avg_merge.plot(x='month', y=['avg_weekday','avg_weekend'],kind='bar',grid=True,xlabel='Month',ylabel='Avg Sales',rot=0,figsize=(10,5))

<Axes: xlabel='Month', ylabel='Avg Sales'>

## `Jawaban No 4.2`

In [ ]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
data42 = {'periode' : 'Total 3 Months',
          'avg_sales_weekend' : weekend['weekend'].mean().round(2),
          'avg_sales_weekday' : weekday['weekday'].mean().round(2),
          'growth' : (weekend['weekend'].mean() - weekday['weekday'].mean()).round(2),
          'percentage' : ((weekend['weekend'].mean() - weekday['weekday'].mean())/weekend['weekend'].mean()*100).round(2)
          }
data42 = pd.DataFrame(data42, index=[0])
data42

periode  avg_sales_weekend  avg_sales_weekday      growth  \
0  Total 3 Months         6147516.67          8522949.2 -2375432.53   

   percentage  
0      -38.64